In [1]:
import numpy as np
import scipy as sp
import smetana
from reframed import Environment
import cobra
from smetana.interface import load_media_db
from smetana.legacy import Community
from smetana.interface import define_environment
from smetana.smetana import minimal_environment



In [2]:
model_paths = {'L_lactis': './models/L_lactis/L_lactis.xml',
                'L_plantarum': './models/L_plantarum/L_plantarum.xml',
                'S_cerevisiae': './models/S_cerevisiae/12918_2012_1082_MOESM2_ESM.XML'}
media_db = load_media_db('./media_db.tsv')


In [4]:
%%capture
l_lactis = cobra.io.read_sbml_model(model_paths['L_lactis'])
l_plant_model = cobra.io.read_sbml_model(model_paths['L_plantarum'])


In [11]:
from smetana.interface import load_communities
from smetana.interface import load_media

model_cache, comm_dict, other_models = load_communities([model_paths['L_lactis'], model_paths['L_plantarum'],], None, None, flavor='cobra')
community = Community(comm_id, comm_models, copy_models=False)
comm_models = [model_cache.get_model(org_id, reset_id=True) for org_id in organisms]
community = Community('all', comm_models, copy_models=False)
compl_env = Environment.complete(community.merged, max_uptake=1.0)
print(compl_env)


R_EX_M_glc__D_e_pool	-10.0	inf
R_EX_M_h2o_e_pool	-10.0	inf
R_EX_M_14glucan_e_pool	-10.0	inf
R_EX_M_26dap__M_e_pool	-10.0	inf
R_EX_M_h_e_pool	-10.0	inf
R_EX_M_2hxmp_e_pool	-10.0	inf
R_EX_M_2pglyc_e_pool	-10.0	inf
R_EX_M_3oxoadp_e_pool	-10.0	inf
R_EX_M_LalaDgluMdap_e_pool	-10.0	inf
R_EX_M_ala__L_e_pool	-10.0	inf
R_EX_M_cl_e_pool	-10.0	inf
R_EX_M_dextrin_e_pool	-10.0	inf
R_EX_M_arab__L_e_pool	-10.0	inf
R_EX_M_abt__L_e_pool	-10.0	inf
R_EX_M_4abut_e_pool	-10.0	inf
R_EX_M_acald_e_pool	-10.0	inf
R_EX_M_ac_e_pool	-10.0	inf
R_EX_M_chol_e_pool	-10.0	inf
R_EX_M_pi_e_pool	-10.0	inf
R_EX_M_ribflv_e_pool	-10.0	inf
R_EX_M_nh4_e_pool	-10.0	inf
R_EX_M_ade_e_pool	-10.0	inf
R_EX_M_mal__L_e_pool	-10.0	inf
R_EX_M_alaala_e_pool	-10.0	inf
R_EX_M_cys__L_e_pool	-10.0	inf
R_EX_M_ala__D_e_pool	-10.0	inf
R_EX_M_gly_e_pool	-10.0	inf
R_EX_M_ser__L_e_pool	-10.0	inf
R_EX_M_arg__L_e_pool	-10.0	inf
R_EX_M_cgly_e_pool	-10.0	inf
R_EX_M_malt_e_pool	-10.0	inf
R_EX_M_amylose300_e_pool	-10.0	inf
R_EX_M_anhgm_e_pool	-10.0	inf

In [8]:
for comm_id, organisms in list(comm_dict.items()):
    comm_models = [model_cache.get_model(org_id, reset_id=True) for org_id in organisms]
    community = Community(comm_id, comm_models, copy_models=False)
    env = minimal_environment(community, True, verbose=True, min_mol_weight=False,
                                use_lp=True, max_uptake=10.0)
    
    compl_env = Environment.complete(community.merged, max_uptake=10.0)
env

R_EX_M_xan_e_pool	-10.0	inf
R_EX_M_ac_e_pool	-10.0	inf
R_EX_M_orn_e_pool	-10.0	inf
R_EX_M_ala__D_e_pool	-10.0	inf
R_EX_M_co2_e_pool	-10.0	inf
R_EX_M_udcpo5_e_pool	-10.0	inf

In [133]:
community.merged.reactions['R_EX_M_ala__D_e_pool'].lb

-inf

In [10]:
comm_dict

{'all': ['L_lactis', 'L_plantarum']}

In [45]:

medium_id = "minimal"


AttributeError: 'Model' object has no attribute 'biomass_reaction'

In [12]:
import pandas as pd
df = pd.read_csv('./carveme_output/lactis_plant_detailed.tsv', delimiter='\t')



In [ ]:
sub_df = df.loc[(df[''])]

In [182]:
# Select for compounds involved in mutualism
df = df.loc[df['smetana'] > 0.0].reset_index(drop=True)
df['clean_compounds'] = [m.replace('M_', '').replace('_e', '') for m in df['compound'].values]



In [183]:
# Load media database
media_db = pd.read_csv('./media_db.tsv', delimiter='\t')
LB_media = media_db.loc[media_db['medium'] == 'LB']

LB_media_no_crossfeed = LB_media.loc[~LB_media['compound'].isin(df['clean_compounds'])]


In [184]:
LB_media_no_crossfeed.to_csv('./media_plant_lactis_db.tsv', sep='\t')

In [192]:
l_lactis.reactions.set_flux_bounds("R_EX_M_h2o_e_pool", -0, 0)
l_lactis.slim_optimize()

AttributeError: DictList has no attribute or entry set_flux_bounds

In [262]:
x='EX_h2o_e'
l_lactis.reactions.EX_h2o_e
l_lactis.reactions.get_by_id('EX_h2o_e').upper_bound = 100.0
l_lactis.reactions.get_by_id('EX_h2o_e').lower_bound = 0
l_lactis.reactions.get_by_id('EX_glyc_e').lower_bound = 0
l_lactis.reactions.get_by_id('EX_glyc_e').upper_bound = 100
l_lactis.reactions.get_by_id('EX_so4_e').lower_bound = 0
l_lactis.reactions.get_by_id('EX_so4_e').upper_bound = np.inf


In [263]:
l_lactis.slim_optimize()
# l_lactis.reactions.get_by_id('EX_h2o_e')

0.0